## THIS NOTEBOOK IS WIP ##

This notebook simply serves to attempt to provide answers to some of the burning questions we all have about the McDonald's menu. For example,

 - How many calories and carbohydrates does a soft drink contribute to a McDonald's meal (this is a recommended question, but I was curious)?
 - How much of each menu item could you eat if you stuck to the recommended serving size?
 - What's the most balanced meal you can eat at McDonald's?
 - Which nutritional values across the menu are most out of line with recommended servings?
 - How does a breakfast meal compare nutritionally to a lunch meal?
 - Are "Mc" food items any more nutritionally significant than non-branded items?
 - What difference does grilled make compared to crispy?

I'm clearly still a newbie at this data stuff, but we all start somewhere. :)

Pandas is a great library for loading and exploring data. In general, I prefer matplotlib over plot.ly for my visualizations.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from cycler import cycler
import re

menu = pd.read_csv('../input/menu.csv')

Always check the quality of data before playing with it, in case of inconsistent columns or null values in some rows.

In [ ]:
print('Table is {0} rows by {1} columns.'.format(menu.shape[0], menu.shape[1]))
print('Are there null vals? \n{0}'.format(menu.isnull().any()))

Let's also take a quick look at the kinds of data we have to work with.

In [ ]:
columnNames = list(menu)
sampleRow = menu.iloc[1]
print('{0:<35}{1}'.format('Column name', 'Example data'))
for i in range(0, len(columnNames)):
    print('{0:<35}{1}'.format(columnNames[i], sampleRow[i]))

print('\nCategories:')
for category in menu.Category.unique():
    print(category)

The data appears to check out. Now we can start answering some questions.
## How many calories/carbs does a drink contribute to a meal? ##
According to McDonald's ["Meal" menu items][1], a meal consists of an entrée, a medium fries or a salad, and a soft drink. Of course, the nutritional makeup of the meal depends on the meal selected, so we'll try a few different ones. 

**Big Mac, medium fries, soft drink**

First, we need to know (without peeking!) how our data source refers to the various Items on offer. Here, I "search" for terms that I'm interested in.

  [1]: http://www.fastfoodmenuprices.com/mcdonalds-prices/

In [ ]:
print('\"Mac\" items:\n{0}'.format(menu[menu['Item'].str.contains("mac", False)]['Item']))
print('\nFries items:\n{0}'.format(menu[menu['Item'].str.contains("fries", False)]['Item']))
print('\nDrink items:\n{0}'.format(menu[menu['Item'].str.contains("drink", False)]['Item']))

It appears that there are no Items with "drink" in their name (neither "soda" nor "soft" work either). It must be the case that the drinks are named specifically. This assumption is further validated by the fact that we now notice both "Beverages" and "Coffee & Tea" categories.

In [ ]:
print('Drink items:\n{0}'.format(menu.loc[menu['Category'] == 'Beverages']['Item']))

Aha, there they are. For now, I'll pick Dr. Pepper, my favorite of the soft drinks on offer. But looking at this list raises a question about the superlatives of each drink (most sugar? fewest calories?). Let's revisit the drinks in a moment.

We are now equipped to answer the question at hand. I'll retrieve and chart the calories and carbohydrates of the meal compared to that of the drink. I'll also add a third set of charts to compare how much substance (in grams) each item takes up in our meal.

In [ ]:
burgerCals = menu.loc[menu['Item'] == 'Big Mac']['Calories']
burgerCarbs = menu.loc[menu['Item'] == 'Big Mac']['Carbohydrates']
burgerMass = menu.loc[menu['Item'] == 'Big Mac']['Serving Size']
friesCals = menu.loc[menu['Item'] == 'Medium French Fries']['Calories']
friesCarbs = menu.loc[menu['Item'] == 'Medium French Fries']['Carbohydrates']
friesMass = menu.loc[menu['Item'] == 'Medium French Fries']['Serving Size']
drinkSCals = menu.loc[menu['Item'] == 'Dr Pepper (Small)']['Calories']
drinkSCarbs = menu.loc[menu['Item'] == 'Dr Pepper (Small)']['Carbohydrates']
drinkSMass = menu.loc[menu['Item'] == 'Dr Pepper (Small)']['Serving Size']
drinkMCals = menu.loc[menu['Item'] == 'Dr Pepper (Medium)']['Calories']
drinkMCarbs = menu.loc[menu['Item'] == 'Dr Pepper (Medium)']['Carbohydrates']
drinkMMass = menu.loc[menu['Item'] == 'Dr Pepper (Medium)']['Serving Size']
drinkLCals = menu.loc[menu['Item'] == 'Dr Pepper (Large)']['Calories']
drinkLCarbs = menu.loc[menu['Item'] == 'Dr Pepper (Large)']['Carbohydrates']
drinkLMass = menu.loc[menu['Item'] == 'Dr Pepper (Large)']['Serving Size']

find = re.compile(r"^([0-9|\.]*)[A-z]*")
calsS = [burgerCals.values, friesCals.values, drinkSCals.values]
calsM = [burgerCals.values, friesCals.values, drinkMCals.values]
calsL = [burgerCals.values, friesCals.values, drinkLCals.values]
carbsS = [burgerCarbs.values, friesCarbs.values, drinkSCarbs.values]
carbsM = [burgerCarbs.values, friesCarbs.values, drinkMCarbs.values]
carbsL = [burgerCarbs.values, friesCarbs.values, drinkLCarbs.values]
massS = [re.search(find, str(burgerMass.values[0])).group(0), re.search(find, str(friesMass.values[0])).group(0), re.search(find, str(drinkSMass.values[0])).group(0)]
massM = [re.search(find, str(burgerMass.values[0])).group(0), re.search(find, str(friesMass.values[0])).group(0), re.search(find, str(drinkMMass.values[0])).group(0)]
massL = [re.search(find, str(burgerMass.values[0])).group(0), re.search(find, str(friesMass.values[0])).group(0), re.search(find, str(drinkLMass.values[0])).group(0)]

print('For your reference:')
print('{0:<20}{1:<15}{2:<15}{3}'.format('', 'Calories', 'Carbohydrates', 'Mass'))
print('{0:<20}{1:<15}{2:<15}{3}'.format('Big Mac', burgerCals.values[0], burgerCarbs.values[0], burgerMass.values[0]))
print('{0:<20}{1:<15}{2:<15}{3}'.format('Medium Fries', friesCals.values[0], friesCarbs.values[0], friesMass.values[0]))
print('{0:<20}{1:<15}{2:<15}{3}'.format('Dr. Pepper (small)', drinkSCals.values[0], drinkSCarbs.values[0], drinkSMass.values[0]))
print('{0:<20}{1:<15}{2:<15}{3}'.format('Dr. Pepper (medium)', drinkMCals.values[0], drinkMCarbs.values[0], drinkMMass.values[0]))
print('{0:<20}{1:<15}{2:<15}{3}'.format('Dr. Pepper (large)', drinkLCals.values[0], drinkLCarbs.values[0], drinkLMass.values[0]))

fig1, axarr = plt.subplots(3, 3, figsize=(12, 12))
explode = (0, 0, 0.1)
labelS = 'Big Mac', 'Medium Fries', 'Dr. Pepper (small)'
labelM = 'Big Mac', 'Medium Fries', 'Dr. Pepper (medium)'
labelL = 'Big Mac', 'Medium Fries', 'Dr. Pepper (large)'

axarr[0, 0].pie(calsS, explode=explode, labels=labelS, autopct='%1.1f%%', startangle=90)
axarr[0, 1].pie(carbsS, explode=explode, labels=labelS, autopct='%1.1f%%', startangle=90)
axarr[0, 2].pie(massS, explode=explode, labels=labelS, autopct='%1.1f%%', startangle=90)
axarr[1, 0].pie(calsM, explode=explode, labels=labelM, autopct='%1.1f%%', startangle=90)
axarr[1, 1].pie(carbsM, explode=explode, labels=labelM, autopct='%1.1f%%', startangle=90)
axarr[1, 2].pie(massM, explode=explode, labels=labelM, autopct='%1.1f%%', startangle=90)
axarr[2, 0].pie(calsL, explode=explode, labels=labelL, autopct='%1.1f%%', startangle=90)
axarr[2, 1].pie(carbsL, explode=explode, labels=labelL, autopct='%1.1f%%', startangle=90)
axarr[2, 2].pie(massL, explode=explode, labels=labelL, autopct='%1.1f%%', startangle=90)

for i in range(0, len(axarr)):
    for j in range(0, len(axarr[0])):
        axarr[i, j].axis('equal')
        axarr[i, j].set_prop_cycle(cycler('color', ['burlywood', 'lemonchiffon', 'red']))
                                   
axarr[0, 0].set_title('Calories with small drink')
axarr[0, 1].set_title('Carbs with small drink')
axarr[0, 2].set_title('Mass (oz) with small drink')
axarr[1, 0].set_title('Calories with medium drink')
axarr[1, 1].set_title('Carbs with medium drink')
axarr[1, 2].set_title('Mass (oz) with medium drink')
axarr[2, 0].set_title('Calories with large drink')
axarr[2, 1].set_title('Carbs with large drink')
axarr[2, 2].set_title('Mass (oz) with large drink')
plt.show()